# Content Repo & FAQ Builder (End‑to‑End)

This notebook creates **two SQLite databases** end‑to‑end:

1. **`content_repo.db`** — sample content records with page numbers, versions, checksums (using your uploaded generator as a base, then extending it with realistic modify/insert/delete scenarios).
2. **`faq_question.db`** — FAQ questions seeded from your spreadsheet and **auto‑generated** from the markdown content.

**Inputs used (already uploaded):**
- `/mnt/data/generate_version_tracking_test_data.py` — base generator for `content_repo.db`
- `/mnt/data/faq_question.xlsx` — sample FAQ rows to seed the `faq_question` table
- (Optional) `/mnt/data/test_version_tracking_scenarios.ipynb` — referenced for process ideas

> Run cells top‑to‑bottom the first time.


In [ ]:
from __future__ import annotations
import os, json, hashlib, sqlite3, runpy, re
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple

import pandas as pd

# Paths
ROOT = Path('.').resolve()
DATA_DIR = ROOT / 'test_data_version_tracking'
MARKDOWN_DIR = DATA_DIR / 'markdown_files'
DB_DIR = ROOT / 'databases'
DB_DIR.mkdir(parents=True, exist_ok=True)

UPLOADED_GENERATOR = Path('/mnt/data/generate_version_tracking_test_data.py')  # provided
FAQ_XLSX = Path('/mnt/data/faq_question.xlsx')  # provided
UPLOADED_HELP_NOTEBOOK = Path('/mnt/data/test_version_tracking_scenarios.ipynb')  # optional

BASE_TIME = datetime(2025, 1, 1, 10, 0, 0)

def get_timestamp(days_offset: int, hours_offset: int = 0) -> str:
    dt = BASE_TIME + timedelta(days=days_offset, hours=hours_offset)
    return dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')

def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode('utf-8')).hexdigest()

def write_text(path: Path, content: str) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(content, encoding='utf-8')


In [ ]:
# Run your uploaded generator (creates ./databases/content_repo.db with initial data)
assert UPLOADED_GENERATOR.exists(), f"Missing: {UPLOADED_GENERATOR}"
_ = runpy.run_path(str(UPLOADED_GENERATOR))

# Validate initial DB existence
CONTENT_DB_PATH = DB_DIR / 'content_repo.db'
assert CONTENT_DB_PATH.exists(), f"content_repo.db not created at {CONTENT_DB_PATH}"
print("Found/created:", CONTENT_DB_PATH)

## Extend Dataset to Cover Modify / Insert / Delete

We add realistic updates beyond the base generator:
- **Day 2 (2025‑01‑02):** Modify Page 2 (same page number, **version +1**, checksum changes)
- **Day 3 (2025‑01‑03):** Insert new Page at position 3 (new content id), **shift old page 3 → 4** with version +1
- **Day 4 (2025‑01‑04):** Mark original Page 2 (**id=1002**) as Inactive (delete); shift others down

In [ ]:
import sqlite3, pandas as pd

conn = sqlite3.connect(CONTENT_DB_PATH)
cur = conn.cursor()

# Ensure tables exist (created by generator)
cur.execute("PRAGMA table_info(content_repo)")
cols = [c[1] for c in cur.fetchall()]
expected = {'ud_source_file_id','domain','service','orgoid','associateoid','raw_file_nme','raw_file_type',
            'raw_file_version_nbr','source_url_txt','parent_location_txt','raw_file_path',
            'extracted_markdown_file_path','extracted_layout_file_path','raw_file_page_nbr','title_nme',
            'breadcrumb_txt','content_tags_txt','version_nbr','content_checksum','file_status',
            'created_dt','last_modified_dt'}
missing = expected - set(cols)
if missing:
    raise RuntimeError(f"content_repo columns missing from base generator: {missing}")

# Optional mirror table with operation tracking
cur.execute("PRAGMA table_info(content_repo_with_operations)")
has_with_ops = bool(cur.fetchall())

def insert_row(base: dict, with_ops: Optional[dict] = None):
    # Insert into content_repo
    cols = [c for c in base.keys()]
    placeholders = ",".join(["?"]*len(cols))
    sql = f"INSERT INTO content_repo ({','.join(cols)}) VALUES ({placeholders})"
    cur.execute(sql, [base[c] for c in cols])

    # Optionally into content_repo_with_operations
    if has_with_ops and with_ops:
        cols2 = [c for c in with_ops.keys()]
        placeholders2 = ",".join(["?"]*len(cols2))
        sql2 = f"INSERT INTO content_repo_with_operations ({','.join(cols2)}) VALUES ({placeholders2})"
        cur.execute(sql2, [with_ops[c] for c in cols2])

# Helper to fetch an existing row by id & timestamp (for cloning baseline fields)
def fetch_latest_by_id(content_id: int) -> dict:
    df = pd.read_sql(f"""
        SELECT * FROM content_repo
        WHERE ud_source_file_id = {content_id}
        ORDER BY last_modified_dt DESC
        LIMIT 1
    """, conn)
    if df.empty:
        raise ValueError(f"No rows for content_id={content_id}")
    return df.iloc[0].to_dict()

# --- Day 2: Modify Page 2 (id=1002), version +1, checksum changes ---
p2_mod = (
    "## 4. Sick Leave (Revised)\n"
    "Employees are entitled to sick leave as follows:\n"
    "- 12 days of paid sick leave per year\n"
    "- Medical certificate required for absences over 2 consecutive days\n"
    "- Unused sick leave carries over up to 5 days\n\n"
    "## 5. Parental Leave\n"
    "- 12 weeks of paid parental leave for primary caregiver\n"
    "- 4 weeks of paid parental leave for secondary caregiver\n"
)
p2_path_v2 = MARKDOWN_DIR / 'leave_policy' / 'page2_v2.md'
write_text(p2_path_v2, p2_mod)
p2_checksum_v2 = sha256_text(p2_mod)

base_1002 = fetch_latest_by_id(1002)
row_day2 = base_1002.copy()
row_day2.update({
    'version_nbr': base_1002['version_nbr'] + 1,
    'content_checksum': p2_checksum_v2,
    'extracted_markdown_file_path': str(p2_path_v2.resolve()),
    'last_modified_dt': get_timestamp(1),
    'file_status': 'Active',
})
insert_row(
    row_day2,
    {**row_day2, 'operation': 'UPDATE', 'operation_date': '2025-01-02'} if has_with_ops else None
)

# --- Day 3: Insert new page at position 3 (id=1004), and shift old page 3 (id=1003) to page 4 w/ version +1 ---
new_p3 = (
    "## 5a. Work From Home Policy\n"
    "- Up to 3 days per week with manager approval\n"
    "- Must maintain availability during core hours (10am–4pm)\n"
    "- Equipment stipends available\n"
)
p3_new_path = MARKDOWN_DIR / 'leave_policy' / 'page3_new_v1.md'
write_text(p3_new_path, new_p3)
p3_new_checksum = sha256_text(new_p3)

# Insert new content id=1004 as page 3 (v1)
base_template = fetch_latest_by_id(1001)  # borrow fixed fields (domain/service/title/etc.)
row_1004_day3 = {
    **{k: base_template[k] for k in [
        'domain','service','orgoid','associateoid','raw_file_nme','raw_file_type',
        'raw_file_version_nbr','source_url_txt','parent_location_txt','raw_file_path',
        'title_nme','breadcrumb_txt','content_tags_txt','created_dt'
    ]},
    'ud_source_file_id': 1004,
    'extracted_markdown_file_path': str(p3_new_path.resolve()),
    'extracted_layout_file_path': 'layout_p3_new.txt',
    'raw_file_page_nbr': 3,
    'version_nbr': 1,
    'content_checksum': p3_new_checksum,
    'file_status': 'Active',
    'last_modified_dt': get_timestamp(2),
}
insert_row(
    row_1004_day3,
    {**row_1004_day3, 'operation': 'INSERT', 'operation_date': '2025-01-03'} if has_with_ops else None
)

# Shift id=1003 from page 3 -> 4, bump version
base_1003 = fetch_latest_by_id(1003)
row_1003_day3 = base_1003.copy()
row_1003_day3.update({
    'raw_file_page_nbr': 4,
    'version_nbr': base_1003['version_nbr'] + 1,
    'last_modified_dt': get_timestamp(2),
    'file_status': 'Active',
})
insert_row(
    row_1003_day3,
    {**row_1003_day3, 'operation': 'UPDATE', 'operation_date': '2025-01-03'} if has_with_ops else None
)

# --- Day 4: Delete id=1002 (mark Inactive) and shift others ---
base_1002_now = fetch_latest_by_id(1002)
row_1002_day4 = base_1002_now.copy()
row_1002_day4.update({
    'file_status': 'Inactive',
    'last_modified_dt': get_timestamp(3),
})
insert_row(
    row_1002_day4,
    {**row_1002_day4, 'operation': 'DELETE', 'operation_date': '2025-01-04'} if has_with_ops else None
)

# Update 1004 (shift to page 2, v+1)
base_1004 = fetch_latest_by_id(1004)
row_1004_day4 = base_1004.copy()
row_1004_day4.update({
    'raw_file_page_nbr': 2,
    'version_nbr': base_1004['version_nbr'] + 1,
    'last_modified_dt': get_timestamp(3),
    'file_status': 'Active',
})
insert_row(
    row_1004_day4,
    {**row_1004_day4, 'operation': 'UPDATE', 'operation_date': '2025-01-04'} if has_with_ops else None
)

# Update 1003 (shift to page 3, v+1)
base_1003_now = fetch_latest_by_id(1003)
row_1003_day4 = base_1003_now.copy()
row_1003_day4.update({
    'raw_file_page_nbr': 3,
    'version_nbr': base_1003_now['version_nbr'] + 1,
    'last_modified_dt': get_timestamp(3),
    'file_status': 'Active',
})
insert_row(
    row_1003_day4,
    {**row_1003_day4, 'operation': 'UPDATE', 'operation_date': '2025-01-04'} if has_with_ops else None
)

conn.commit()

summary = pd.read_sql("SELECT ud_source_file_id, raw_file_page_nbr, version_nbr, file_status, last_modified_dt FROM content_repo ORDER BY last_modified_dt, ud_source_file_id", conn)
print("Extended entries (chronological):")
print(summary.to_string(index=False))
conn.close()

In [ ]:
import sqlite3, pandas as pd

conn = sqlite3.connect(CONTENT_DB_PATH)

print("\nLatest entry per content_id:")
latest = pd.read_sql(
    """
    SELECT * FROM content_repo
    WHERE (ud_source_file_id, last_modified_dt) IN (
        SELECT ud_source_file_id, MAX(last_modified_dt)
        FROM content_repo
        GROUP BY ud_source_file_id
    )
    ORDER BY raw_file_page_nbr
    """, conn)
display(latest[['ud_source_file_id','raw_file_page_nbr','version_nbr','file_status','content_checksum']])

print("\nAll entries for id=1002 (shows modify then delete):")
display(pd.read_sql("SELECT * FROM content_repo WHERE ud_source_file_id=1002 ORDER BY last_modified_dt", conn)[['ud_source_file_id','raw_file_page_nbr','version_nbr','file_status','last_modified_dt']])

print("\nAll entries for id=1003 and 1004 (shift behavior across days):")
display(pd.read_sql("SELECT * FROM content_repo WHERE ud_source_file_id IN (1003,1004) ORDER BY ud_source_file_id, last_modified_dt", conn)[['ud_source_file_id','raw_file_page_nbr','version_nbr','file_status','last_modified_dt']])

conn.close()

## Create & Populate `faq_question.db`

Seed from your spreadsheet and auto‑generate from markdown. We preserve source metadata when available.


In [ ]:
import sqlite3, pandas as pd, re

FAQ_DB_PATH = DB_DIR / 'faq_question.db'

# Recreate for idempotency
if FAQ_DB_PATH.exists():
    FAQ_DB_PATH.unlink()

conn_faq = sqlite3.connect(FAQ_DB_PATH)
cur_faq = conn_faq.cursor()

cur_faq.execute("""
CREATE TABLE IF NOT EXISTS faq_question (
  faq_id INTEGER PRIMARY KEY AUTOINCREMENT,
  question_text TEXT NOT NULL,
  answer_text TEXT,
  tags TEXT,
  source_type TEXT,              -- 'uploaded_excel' | 'auto_markdown'
  source_file TEXT,              -- path of markdown or 'faq_question.xlsx'
  ud_source_file_id INTEGER,     -- nullable for spreadsheet-only rows
  raw_file_page_nbr INTEGER,
  version_nbr INTEGER,
  content_checksum TEXT,
  created_dt TEXT DEFAULT CURRENT_TIMESTAMP
)
""")
conn_faq.commit()

print("Created:", FAQ_DB_PATH)

In [ ]:
def normalize_faq_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = {c.lower().strip(): c for c in df.columns}
    # Try to infer question/answer/tags
    q_col = next((cols[k] for k in cols if k in ['question','q','prompt','question_text']), None)
    a_col = next((cols[k] for k in cols if k in ['answer','a','response','answer_text']), None)
    t_col = next((cols[k] for k in cols if k in ['tags','tag','category','topic']), None)

    # If no clear question column, pick first text-like
    if q_col is None:
        q_col = df.columns[0]
    # If no answer, create empty
    if a_col is None:
        df['__answer__'] = ''
        a_col = '__answer__'
    # If no tags, create empty
    if t_col is None:
        df['__tags__'] = ''
        t_col = '__tags__'

    out = df[[q_col, a_col, t_col]].copy()
    out.columns = ['question_text','answer_text','tags']
    # Basic cleaning
    out['question_text'] = out['question_text'].astype(str).str.strip()
    out['answer_text']   = out['answer_text'].astype(str).str.strip()
    out['tags']          = out['tags'].astype(str).str.strip()
    out = out[out['question_text'] != '']
    return out

if FAQ_XLSX.exists():
    df_excel = pd.read_excel(FAQ_XLSX)
    df_norm  = normalize_faq_columns(df_excel)
    df_norm['source_type']  = 'uploaded_excel'
    df_norm['source_file']  = str(FAQ_XLSX)
    df_norm['ud_source_file_id'] = None
    df_norm['raw_file_page_nbr'] = None
    df_norm['version_nbr'] = None
    df_norm['content_checksum'] = None
    df_norm.to_sql('faq_question', conn_faq, if_exists='append', index=False)
    print(f"Ingested {len(df_norm)} rows from spreadsheet.")
else:
    print("No spreadsheet found to ingest.")

In [ ]:
# We parse headings + bullet lists to synthesize Q/A.
# Q types:
#  - "What does <Section Title> cover?"
#  - If a numbered process exists, create "What is the <Section> process?"

def read_all_latest_entries() -> pd.DataFrame:
    conn = sqlite3.connect(CONTENT_DB_PATH)
    df = pd.read_sql(
        """
        SELECT * FROM content_repo
        WHERE (ud_source_file_id, last_modified_dt) IN (
            SELECT ud_source_file_id, MAX(last_modified_dt)
            FROM content_repo
            GROUP BY ud_source_file_id
        )
        ORDER BY raw_file_page_nbr
        """, conn)
    conn.close()
    return df

def extract_sections(md_text: str):
    """
    Return list of (section_title, section_body) based on '## ' headers.
    """
    lines = md_text.splitlines()
    sections = []
    current_title = None
    current_body = []
    for ln in lines:
        if re.match(r'^##\s+', ln):
            if current_title is not None:
                sections.append((current_title.strip('# ').strip(), '\n'.join(current_body).strip()))
                current_body = []
            current_title = ln
        else:
            current_body.append(ln)
    if current_title is not None:
        sections.append((current_title.strip('# ').strip(), '\n'.join(current_body).strip()))
    return sections

def summarize_bullets(text: str) -> str:
    lines = [l.strip() for l in text.splitlines() if l.strip().startswith(('-', '*', '1.', '2.', '3.', '4.', '5.', '6.', '7.'))]
    if not lines:
        parts = re.split(r'(?<=[.!?])\s+', text.strip())
        return ' '.join(parts[:2]).strip()
    return '\n'.join(lines[:8])

def generate_qa_for_markdown(md_path: Path):
    text = md_path.read_text(encoding='utf-8')
    qa_pairs = []
    for title, body in extract_sections(text):
        q1 = f"What does the '{title}' section cover?"
        a1 = summarize_bullets(body) or body[:500]
        qa_pairs.append((q1, a1))

        if re.search(r'process', title, re.I) or re.search(r'^\d+\.', body, re.M):
            q2 = f"What is the {title} process?"
            a2 = summarize_bullets(body)
            qa_pairs.append((q2, a2))
    return qa_pairs

latest = read_all_latest_entries()

auto_rows = []
for _, row in latest.iterrows():
    md_file = Path(row['extracted_markdown_file_path'])
    if not md_file.exists():
        continue
    for q, a in generate_qa_for_markdown(md_file):
        auto_rows.append({
            'question_text': q,
            'answer_text': a,
            'tags': 'policy;auto',
            'source_type': 'auto_markdown',
            'source_file': str(md_file),
            'ud_source_file_id': int(row['ud_source_file_id']),
            'raw_file_page_nbr': int(row['raw_file_page_nbr']),
            'version_nbr': int(row['version_nbr']),
            'content_checksum': row['content_checksum'],
        })

df_auto = pd.DataFrame(auto_rows)
if not df_auto.empty:
    df_auto.to_sql('faq_question', conn_faq, if_exists='append', index=False)
print(f"Auto-generated {len(df_auto)} FAQ rows from markdown.")

In [ ]:
# Deduplicate by normalized question text (case/whitespace)
df_all = pd.read_sql("SELECT * FROM faq_question ORDER BY faq_id", conn_faq)

def norm_q(s: str) -> str:
    return re.sub(r'\s+', ' ', s.strip().lower())

df_all['__norm_q__'] = df_all['question_text'].apply(norm_q)
dupes = df_all[df_all.duplicated('__norm_q__', keep='first')]

if not dupes.empty:
    ids_to_keep = set(df_all.drop_duplicates('__norm_q__', keep='first')['faq_id'])
    ids_to_delete = [int(x) for x in df_all['faq_id'].tolist() if x not in ids_to_keep]
    if ids_to_delete:
        qmarks = ",".join("?"*len(ids_to_delete))
        conn_faq.execute(f"DELETE FROM faq_question WHERE faq_id IN ({qmarks})", ids_to_delete)
        conn_faq.commit()
        print(f"Removed {len(ids_to_delete)} duplicate questions.")
else:
    print("No duplicates to remove.")

# Indexes for speed
conn_faq.execute("CREATE INDEX IF NOT EXISTS idx_faq_question_normq ON faq_question(question_text)")
conn_faq.execute("CREATE INDEX IF NOT EXISTS idx_faq_source ON faq_question(source_type, ud_source_file_id)")
conn_faq.commit()

In [ ]:
print("FAQ counts by source_type:")
display(pd.read_sql("SELECT source_type, COUNT(*) AS n FROM faq_question GROUP BY source_type", conn_faq))

print("Sample FAQs (top 10):")
display(pd.read_sql("SELECT faq_id, question_text, substr(answer_text,1,140)||'…' as answer_preview, source_type FROM faq_question ORDER BY faq_id LIMIT 10", conn_faq))

print("Linkage to content (top 10 with content ids):")
display(pd.read_sql("SELECT faq_id, ud_source_file_id, raw_file_page_nbr, version_nbr FROM faq_question WHERE ud_source_file_id IS NOT NULL LIMIT 10", conn_faq))

conn_faq.close()